In [23]:
# usar MLPClassifier no sklearn
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold # Import Kfold
from sklearn.model_selection import cross_val_score
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt 
from datetime import datetime
PATH_CLASSES="./train_reduced_file_classes.csv"
PATH_BALEIA="./baleiaExtraido.csv"
RANDOM = 40028922

In [24]:
classes = pd.read_csv(PATH_CLASSES)
df = pd.read_csv(PATH_BALEIA)
df['classes'] = classes['classes'].replace(['noise', 'whale'],[0, 1])

# Tratamento
Selecionando apenas as colunas que realmente possuem algum impacto no valor da classe, para melhorar a eficiência de tempo de nosso algortimo.

In [3]:
from scipy import stats
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import pointbiserialr

In [ ]:
if 'dfBi' in locals() : del dfBi     
y = df['classes']
print(f'Antes de remover colunas irrelevantes: {df.shape}')
for colName in df.drop(['classes'], axis=1).columns[1:]:
    #calculate point-biserial correlation
    x = df[colName]
    corr, pVal = pointbiserialr(x, y)
    dfTemp = {'column': colName, 
            'correlation': corr, 
            'pVal': pVal}
    if 'dfBi' in locals():
        #dfBi = pd.concat([dfBi, dfTemp])
        dfBi = pd.concat([dfBi, pd.DataFrame(dfTemp, index=[0])], ignore_index=True)
    else:
        dfBi=pd.DataFrame(dfTemp, index=[0])
        
dfBi = dfBi.loc[dfBi['pVal'] < 0.5].sort_values(by='correlation', ascending=False)
df = df[dfBi['column'].values]
df['classes'] = y
print(f'Depois de remover colunas irrelevantes: {df.shape}')

# Descrição do DF
As colunas são características de áudio e a classes é 0 se é noise e 1 se é baleia

In [31]:
df.describe()

,'Power Spectrum Overall Average70','Power Spectrum Overall Average69','Power Spectrum Overall Standard Deviation147','Power Spectrum Overall Standard Deviation169','Power Spectrum Overall Standard Deviation170','Power Spectrum Overall Standard Deviation171','Power Spectrum Overall Standard Deviation172','Power Spectrum Overall Standard Deviation173','Power Spectrum Overall Standard Deviation228','Power Spectrum Overall Standard Deviation225',...,'Power Spectrum Overall Standard Deviation14','Power Spectrum Overall Standard Deviation6','Power Spectrum Overall Standard Deviation10','Power Spectrum Overall Average14','Power Spectrum Overall Average33','Power Spectrum Overall Average11','Power Spectrum Overall Average34','Power Spectrum Overall Average10','Power Spectrum Overall Average13',classes
count,4.714900e+04,4.714900e+04,4.714900e+04,4.714900e+04,4.714900e+04,4.714900e+04,4.714900e+04,4.714900e+04,4.714900e+04,4.714900e+04,...,47149.000000,47149.000000,47149.000000,47149.000000,4.714900e+04,47149.000000,4.714900e+04,47149.000000,47149.000000,47149.000000
mean,8.483743e-11,8.483743e-11,8.059556e-10,6.574901e-10,6.574901e-10,6.574901e-10,6.574901e-10,6.574901e-10,3.817684e-10,3.817684e-10,...,0.000005,0.000176,0.000021,0.000004,6.174044e-08,0.000014,7.209061e-08,0.000018,0.000007,0.111434
std,9.210431e-09,9.210431e-09,3.383312e-08,3.019254e-08,3.019254e-08,3.019254e-08,3.019254e-08,3.019254e-08,2.059248e-08,2.059248e-08,...,0.000013,0.000771,0.000052,0.000011,3.637188e-07,0.000029,3.497676e-07,0.000040,0.000014,0.314672
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000003,0.000001,0.000000,0.000000e+00,0.000001,0.000000e+00,0.000001,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000002,0.000022,0.000005,0.000001,0.000000e+00,0.000003,0.000000e+00,0.000004,0.000002,0.000000
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000003,0.000040,0.000010,0.000002,0.000000e+00,0.000007,0.000000e+00,0.000009,0.000003,0.000000
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000005,0.000086,0.000020,0.000004,0.000000e+00,0.000013,0.000000e+00,0.000017,0.000006,0.000000
max,1.000000e-06,1.000000e-06,3.000000e-06,3.000000e-06,3.000000e-06,3.000000e-06,3.000000e-06,3.000000e-06,2.000000e-06,2.000000e-06,...,0.001250,0.066600,0.004238,0.001381,3.600000e-05,0.002756,2.800000e-05,0.003945,0.001168,1.000000


In [35]:
df['classes'].value_counts()

0    41895
1     5254
Name: classes, dtype: int64

# Train Test
Faremos uma kfold stratificada para tentar manter as classes balanceadas

In [5]:
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler

In [6]:
X = df.drop(['classes'], axis=1) # Features
y = df['classes'] # Target variable
over_sampler = RandomOverSampler(random_state=RANDOM)
X,y= over_sampler.fit_resample(X, y)
skf = StratifiedKFold(n_splits=10, random_state=RANDOM,shuffle=True)

In [7]:
for train_index, test_index in skf.split(X, y):
    print(y[train_index].value_counts())

1    37706
0    37705
Name: classes, dtype: int64
1    37706
0    37705
Name: classes, dtype: int64
1    37706
0    37705
Name: classes, dtype: int64
1    37706
0    37705
Name: classes, dtype: int64
1    37706
0    37705
Name: classes, dtype: int64
0    37706
1    37705
Name: classes, dtype: int64
0    37706
1    37705
Name: classes, dtype: int64
0    37706
1    37705
Name: classes, dtype: int64
0    37706
1    37705
Name: classes, dtype: int64
0    37706
1    37705
Name: classes, dtype: int64


# BAGGING

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification
import time

In [ ]:
for max_features_val in [0.5,1.0]:
    for estimator in [DecisionTreeClassifier(),KNeighborsClassifier(n_neighbors=3),GaussianNB()]:
        for n_estimator_val in [10,15,20]:
            start = time.time()
            bgclassifier = BaggingClassifier(base_estimator=estimator,
                                             n_estimators=n_estimator_val,
                                             max_features=max_features_val
                                            )
            y_pred = cross_val_predict(bgclassifier,X, y, cv=skf)
            # Model Accuracy
            acuracia = metrics.accuracy_score(y, y_pred)
            acuracia = round(acuracia,3)

            end = time.time()
            elapsed_time = end - start
            elapsed_time = round(elapsed_time ,3)
            clear = lambda: os.system('clear')
            clear()
            arquivo = open("./baggingSaida.txt", "a")
            print(f'{max_features_val}-----"{estimator}"-----{n_estimator_val}-----{elapsed_time}-----{acuracia}')
            arquivo.write(f'{datetime.now()},{max_features_val},"{estimator}",{n_estimator_val},{acuracia},{elapsed_time}\n')
            arquivo.close()
            

0.5-----"DecisionTreeClassifier()"-----10-----159.69-----0.988
0.5-----"DecisionTreeClassifier()"-----15-----248.955-----0.987
0.5-----"DecisionTreeClassifier()"-----20-----475.721-----0.992
0.5-----"KNeighborsClassifier(n_neighbors=3)"-----10-----681.966-----0.953
0.5-----"KNeighborsClassifier(n_neighbors=3)"-----15-----812.504-----0.956
0.5-----"KNeighborsClassifier(n_neighbors=3)"-----20-----1003.325-----0.966
0.5-----"GaussianNB()"-----10-----36.079-----0.527
0.5-----"GaussianNB()"-----15-----56.888-----0.527
0.5-----"GaussianNB()"-----20-----76.462-----0.527
1.0-----"DecisionTreeClassifier()"-----10-----362.347-----0.981
1.0-----"DecisionTreeClassifier()"-----15-----537.279-----0.978
1.0-----"DecisionTreeClassifier()"-----20-----19338.363-----0.983
1.0-----"KNeighborsClassifier(n_neighbors=3)"-----10-----942.394-----0.887


# BOOSTING

In [10]:
from sklearn.ensemble import AdaBoostClassifier
for estimator in [DecisionTreeClassifier(),GaussianNB()]:
    for n_estimator_val in [10,15,20]:
        try:
            start = time.time()
            adaclassifier = AdaBoostClassifier(base_estimator=estimator , 
                                               n_estimators=n_estimator_val)
            y_pred = cross_val_predict(adaclassifier,X, y, cv=skf)
            # Model Accuracy
            acuracia = metrics.accuracy_score(y, y_pred)
            acuracia = round(acuracia,3)

            end = time.time()
            elapsed_time = end - start
            elapsed_time = round(elapsed_time ,3)

            arquivo = open("./boostingSaida.txt", "a")
            print(f'"{estimator}"-----{n_estimator_val}-----{acuracia}-----{elapsed_time}')
            arquivo.write(f'{datetime.now()},"{estimator}",{n_estimator_val},{acuracia},{elapsed_time}\n')
            arquivo.close()
        except:
            pass

"DecisionTreeClassifier()"-----10-----0.951-----48.682
"DecisionTreeClassifier()"-----15-----0.95-----47.697
"DecisionTreeClassifier()"-----20-----0.95-----47.35
"GaussianNB()"-----10-----0.519-----107.594
"GaussianNB()"-----15-----0.525-----156.293
"GaussianNB()"-----20-----0.518-----211.986


# Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier
for criterion_val in ['gini', 'entropy', 'log_loss']:
    for n_estimator_val in [10,15,20]:
        try:
            start = time.time()
            rf = RandomForestClassifier(n_estimators=n_estimator_val,
                                        criterion=criterion_val)

            y_pred = cross_val_predict(rf,X, y, cv=skf)
            # Model Accuracy
            acuracia = metrics.accuracy_score(y, y_pred)
            acuracia = round(acuracia,3)

            end = time.time()
            elapsed_time = end - start
            elapsed_time = round(elapsed_time ,3)

            arquivo = open("./randomForestSaida.txt", "a")
            print(f'{criterion_val}-----{n_estimator_val}-----{acuracia}-----{elapsed_time}')
            arquivo.write(f'{datetime.now()},{criterion_val},{n_estimator_val},{acuracia},{elapsed_time}\n')
            arquivo.close()
        except:
            pass

gini-----10-----0.989-----31.76
gini-----15-----0.986-----45.441
gini-----20-----0.991-----59.866
entropy-----10-----0.988-----34.141
entropy-----15-----0.986-----50.471
entropy-----20-----0.991-----67.296
log_loss-----10-----0.988-----34.244
log_loss-----15-----0.987-----50.144
log_loss-----20-----0.991-----74.661
